In [11]:
import os
import glob
from collections import defaultdict

class FMRIPrepParser:
    """
    Parser for fMRIPrep BIDS-Derivative directory.

    Attributes
    ----------
    root : str
        Path to the fMRIPrep derivatives root.
    subjects : list of str
        List of subject labels (e.g., ['sub-01', 'sub-02']).
    sessions : dict
        Mapping from subject label to list of session labels (e.g., {'sub-01': ['ses-01', 'ses-02']}).
    """

    def __init__(self, root):
        """
        Initialize parser, discover subjects and their sessions.
        
        Parameters
        ----------
        root : str
            Path to the fMRIPrep derivatives directory.
        """
        self.root = os.path.abspath(root)
        # Discover subject directories: sub-<label>
        subj_pattern = os.path.join(self.root, 'sub-*')
        subjs = sorted(glob.glob(subj_pattern))
        self.subjects = [os.path.basename(s) for s in subjs]  # e.g., 'sub-01' :contentReference[oaicite:5]{index=5}
        self.subjects = [s for s in self.subjects if not s.endswith('.html')]
        
        
        
    
        
        # Discover sessions per subject
        self.sessions = {}
        for subj in self.subjects:
            subj_dir = os.path.join(self.root, subj)
            # session directories: ses-<label>/anat or ses-<label>/func
            ses_pattern = os.path.join(subj_dir, 'ses-*')
            ses_dirs = sorted(glob.glob(ses_pattern))
            if ses_dirs:
                # extract 'ses-XX'
                self.sessions[subj] = [os.path.basename(s) for s in ses_dirs]  # :contentReference[oaicite:6]{index=6}
            else:
                # no session folders => single implicit session
                self.sessions[subj] = [None]

    def _make_pattern(self, subj, ses, suffix_parts):
        """
        Helper to build a glob pattern for BIDS files.
        """
        parts = [self.root, subj]
        if ses:
            parts.append(ses)
        # functional/anat folder based on suffix
        if 'bold' in suffix_parts or 'confounds' in suffix_parts or 'events' in suffix_parts:
            parts.append('func')
        else:
            parts.append('anat')
        # join and append file pattern
        base = os.path.join(*parts)
        # e.g., sub-01[_ses-XX]_task-*_run-*_bold*.nii.gz
        pat = base + suffix_parts
        return pat

    def get_epi(self, subject, session=None, task=None, run=None, space='*'):
        """
        Retrieve preprocessed BOLD (EPI) file paths.
        
        Returns
        -------
        list of str
            Matching .nii.gz file paths.
        """
        subj = subject
        ses = session
        # build BIDS pattern
        fmt = '_'.join(filter(None, [
            subj,
            f"{ses}" if ses else None,
            f"task-{task}" if task else None,
            f"run-{run}" if run else None,
            f"space-{space}"
        ]))
        # suffix pattern: *_bold.nii.gz
        suffix = f"{fmt}_bold.nii.gz"
        pat = self._make_pattern(subj, ses, '/' + suffix)
        return sorted(glob.glob(pat))

    def get_confounds(self, subject, session=None, task=None, run=None, space='*'):
        """
        Retrieve confounds TSV file paths.
        
        Returns
        -------
        list of str
            Matching *confounds.tsv files.
        """
        subj = subject
        ses = session
        fmt = '_'.join(filter(None, [
            subj,
            f"{ses}" if ses else None,
            f"task-{task}" if task else None,
            f"run-{run}" if run else None,
            f"space-{space}"
        ]))
        suffix = f"{fmt}_desc-confounds_regressors.tsv"
        pat = self._make_pattern(subj, ses, '/' + suffix)
        return sorted(glob.glob(pat))  # :contentReference[oaicite:7]{index=7}

    def get_anat(self, subject, session=None, space='T1w'):
        """
        Retrieve anatomical (T1w) image paths.
        
        Returns
        -------
        list of str
            Matching T1w .nii.gz files.
        """
        subj = subject
        ses = session
        fmt = '_'.join(filter(None, [
            subj,
            f"{ses}" if ses else None,
            f"{space}"
        ]))
        suffix = f"{fmt}.nii.gz"
        pat = self._make_pattern(subj, ses, '/' + suffix)
        return sorted(glob.glob(pat))  # :contentReference[oaicite:8]{index=8}

    def get_events(self, subject, session=None, task=None, run=None):
        """
        Retrieve BIDS events TSV file paths.
        
        Returns
        -------
        list of str
            Matching *_events.tsv files.
        """
        subj = subject
        ses = session
        fmt = '_'.join(filter(None, [
            subj,
            f"{ses}" if ses else None,
            f"task-{task}" if task else None,
            f"run-{run}" if run else None
        ]))
        suffix = f"{fmt}_events.tsv"
        pat = self._make_pattern(subj, ses, '/' + suffix)
        return sorted(glob.glob(pat))  # :contentReference[oaicite:9]{index=9}


In [21]:
parser = FMRIPrepParser('../Data/020-fmriprepped')
#print(parser.subjects)                  # ['sub-01', 'sub-02', ...]
sub = parser.subjects[0]
print(parser.sessions[sub])        # ['ses-01', 'ses-02'] or [None] if no sessions
sess = parser.sessions[sub][0]
epis = parser.get_epi(subject=sub, session=sess, task=None, run='1')
#conf = parser.get_confounds('sub-01', session='ses-01', task='localizer', run='1')
#anat = parser.get_anat('sub-01')
#events = parser.get_events('sub-01', session='ses-01', task='localizer', run='1')


['ses-baselineYear1Arm1']


In [25]:
parser.sessions

{'sub-NDARINV1H7JEJW1': ['ses-baselineYear1Arm1'],
 'sub-NDARINV29P0F670': ['ses-baselineYear1Arm1'],
 'sub-NDARINV3385AZ13': ['ses-baselineYear1Arm1'],
 'sub-NDARINV5YHWBYRD': ['ses-baselineYear1Arm1'],
 'sub-NDARINV6WXU5DEY': ['ses-baselineYear1Arm1'],
 'sub-NDARINV8RN0AJRP': ['ses-baselineYear1Arm1'],
 'sub-NDARINVE1XNAPC7': ['ses-baselineYear1Arm1'],
 'sub-NDARINVET4W53Z8': ['ses-baselineYear1Arm1'],
 'sub-NDARINVG0K2DRXF': ['ses-baselineYear1Arm1'],
 'sub-NDARINVHPU5TKNF': ['ses-baselineYear1Arm1'],
 'sub-NDARINVHRTU9HVX': ['ses-baselineYear1Arm1'],
 'sub-NDARINVJNDY0DWK': ['ses-baselineYear1Arm1'],
 'sub-NDARINVJPMRY00B': ['ses-baselineYear1Arm1'],
 'sub-NDARINVKK5BJGB6': ['ses-baselineYear1Arm1'],
 'sub-NDARINVMMN463FT': ['ses-baselineYear1Arm1'],
 'sub-NDARINVN9D4XZKE': ['ses-baselineYear1Arm1'],
 'sub-NDARINVNNNX0496': ['ses-baselineYear1Arm1'],
 'sub-NDARINVPAJ26DUR': ['ses-baselineYear1Arm1'],
 'sub-NDARINVPURW8L0G': ['ses-baselineYear1Arm1'],
 'sub-NDARINVR6R9DHFF': ['ses-b